In [8]:
import numpy as np
import pandas as pd
import random
import os
import seaborn as sns; sns.set_theme(color_codes=True)

from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import  TensorDataset, DataLoader

import pickle

import copy
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix,recall_score,precision_score,precision_recall_curve,f1_score,auc
from sklearn.preprocessing import minmax_scale
from sklearn.svm import SVC
from sklearn.metrics import r2_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import  mean_squared_error
import scipy.stats as ss

random_seed = 0
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

torch.cuda.set_device(3)
os.environ['CUDA_VISIBLE_DEVICES']='3'

In [9]:
class DNN4(nn.Module):
    def __init__(self):
        super(DNN4, self).__init__()
        self.dense1 =nn.Sequential(nn.Linear(14040 , int(14040 *1/32)))
        self.dense2 =nn.Sequential(nn.Linear(int(14040 *1/32), int(14040 *1/512)))
        self.dense4  =nn.Sequential(nn.Linear(int(14040 *1/512),int(14040 *1/8192)))
        self.dense6 =nn.Sequential(nn.Linear(int(14040 *1/8192),1),nn.Sigmoid())

    def forward(self, X):
        X = self.dense1(X)
        X = self.dense2(X)
        X = self.dense4(X)             
        se_predict = self.dense6(X)

        return se_predict

In [10]:
class DNN6(nn.Module):
    def __init__(self):
        super(DNN6, self).__init__()
        self.dense1 =nn.Sequential(nn.Linear(14040 , int(14040 *1/16)))
        self.dense2 =nn.Sequential(nn.Linear(int(14040 *1/16), int(14040 *1/64)))
        self.dense3 =nn.Sequential(nn.Linear(int(14040 *1/64),int(14040 *1/256)))
        self.dense4  =nn.Sequential(nn.Linear(int(14040 *1/256),int(14040 *1/1024)))
        self.dense5  =nn.Sequential(nn.Linear(int(14040 *1/1024),int(14040 *1/4096)))
        self.dense6 =nn.Sequential(nn.Linear(int(14040 *1/4096),1),nn.Sigmoid())

    def forward(self, X):
        X = self.dense1(X)
        X = self.dense2(X)
        X = self.dense3(X)
        X = self.dense4(X)        
        X = self.dense5(X)        
        se_predict = self.dense6(X)

        return se_predict

In [11]:
class DNN8(nn.Module):
    def __init__(self):
        super(DNN8, self).__init__()
        self.dense0 =nn.Sequential(nn.Linear(14040 , int(14040 *1/2)))
        self.dense1 =nn.Sequential(nn.Linear(int(14040 *1/2) , int(14040 *1/8)))
        self.dense12 =nn.Sequential(nn.Linear(int(14040 *1/8) , int(14040 *1/32)))
        self.dense2 =nn.Sequential(nn.Linear(int(14040 *1/32), int(14040 *1/128)))
        self.dense3 =nn.Sequential(nn.Linear(int(14040 *1/128),int(14040 *1/512)))
        self.dense4  =nn.Sequential(nn.Linear(int(14040 *1/512),int(14040 *1/2048)))
        self.dense5  =nn.Sequential(nn.Linear(int(14040 *1/2048),int(14040 *1/8192)))
        self.dense6 =nn.Sequential(nn.Linear(int(14040 *1/8192),1),nn.Sigmoid())

    def forward(self, X):
        X = self.dense0(X)
        X = self.dense1(X)
        X = self.dense12(X)
        X = self.dense2(X)
        X = self.dense3(X)
        X = self.dense4(X)        
        X = self.dense5(X)        
        se_predict = self.dense6(X)

        return se_predict

In [12]:
ANM1 = pd.read_csv('/user_home/moons/EW/official/ANM1_inter_norm.csv',sep=',',index_col=0)
ANM2 = pd.read_csv('/user_home/moons/EW/official/ANM2_inter_norm.csv',sep=',',index_col=0)
ADNI = pd.read_csv('/user_home/moons/EW/official/ADNI_inter_norm.csv',sep=',',index_col=0)
add1 = pd.read_csv('/user_home/moons/EW/official/EW_inter_norm.csv',sep=',',index_col=0)
ROS = pd.read_csv('/user_home/moons/EW/official/ROSMAP_inter_norm.csv',sep=',',index_col=0)

/user_home/moons/anaconda3/envs/pytorch/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,

In [13]:
add1 = add1.loc[:,add1.loc['DX']!='MCI' ]
annot = add1.loc['DX'].replace('NL' ,0)
annot = annot.replace('AD' ,1)
ad_ge_EW = np.ravel(annot, order='F').astype('float32')
EW_SNP = add1.loc['APOE4']
EW_MMSE= add1.loc['MMSE']
EW_value = add1
EW_value.drop('APOE4',axis=0,inplace=True)
EW_value.drop('DX',axis=0,inplace=True)
EW_value.drop('MMSE',axis=0,inplace=True)


ADNI = ADNI.loc[:,ADNI.loc['DX']!='MCI' ]
ADNI = ADNI.dropna(axis=1)
annot = ADNI.loc['DX'].replace('NL' ,0)
annot = annot.replace('AD' ,1)
ad_ge_ADNI = np.ravel(annot, order='F').astype('float32')
ADNI_SNP = ADNI.loc['APOE4']
ADNI_MMSE= ADNI.loc['MMSE']        
ADNI.drop('APOE4',axis=0,inplace=True)
ADNI.drop('DX',axis=0,inplace=True)
ADNI.drop('MMSE',axis=0,inplace=True)
ADNI_value = ADNI

ANM1 = ANM1.loc[:,ANM1.loc['DX']!='MCI' ]
annot = ANM1.loc['DX'].replace('NL' ,0)
annot = annot.replace('AD' ,1)
ad_ge_ANM1 = np.ravel(annot, order='F').astype('float32')
ANM1_SNP = ANM1.loc['APOE4']
ANM1_MMSE= ANM1.loc['MMSE']
ANM1.drop('APOE4',axis=0,inplace=True)
ANM1.drop('MMSE',axis=0,inplace=True)
ANM1.drop('DX',axis=0,inplace=True)
ANM1_value = ANM1

ANM2 = ANM2.loc[:,ANM2.loc['DX']!='MCI' ]
annot = ANM2.loc['DX'].replace('NL' ,0)
annot = annot.replace('AD' ,1)
ad_ge_ANM2 = np.ravel(annot, order='F').astype('float32')
ANM2.drop('DX',axis=0,inplace=True)
ANM2.loc['AGE'] = pd.DataFrame(ANM2.loc['AGE']).fillna(pd.DataFrame(ANM2.loc['AGE']).astype('float').mean(axis=0)).T.loc['AGE']
ANM2_SNP = ANM2.loc['APOE4']
ANM2_MMSE= ANM2.loc['MMSE']
ANM2.drop('APOE4',axis=0,inplace=True)
ANM2.drop('MMSE',axis=0,inplace=True)
ANM2_value = ANM2

ROS = ROS.loc[:,ROS.loc['DX']!='MCI' ]
annot = ROS.loc['DX'].replace('NL' ,0)
annot = annot.replace('AD' ,1)

ad_ge_ROS = np.ravel(annot, order='F').astype('float32')
ROS_SNP = ROS.loc['APOE4']
ROS_MMSE= ROS.loc['MMSE']
ROS.drop('APOE4',axis=0,inplace=True)
ROS.drop('MMSE',axis=0,inplace=True)
ROS.drop('DX',axis=0,inplace=True)
ROS_value = ROS

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
yg_train = np.concatenate([ad_ge_ADNI,ad_ge_ANM1,ad_ge_ANM2,ad_ge_ROS])
select_X_train=  np.concatenate([ADNI_value.T.values,ANM1_value.T.values,ANM2_value.T.values,ROS_value.T.values])
SNP_train =np.concatenate([ADNI_SNP.values.reshape(-1,1),ANM1_SNP.values.reshape(-1,1),ANM2_SNP.values.reshape(-1,1), ROS_SNP.values.reshape(-1,1)])
y_MMSE_train =np.concatenate([ADNI_MMSE.values.reshape(-1,1),ANM1_MMSE.values.reshape(-1,1),ANM2_MMSE.values.reshape(-1,1), ROS_MMSE.values.reshape(-1,1)])

select_X_test= EW_value.T.values
yg_test= ad_ge_EW
SNP_test= EW_SNP.values.reshape(-1,1)
y_MMSE_test=  EW_MMSE.values.astype('float').reshape(-1,1)

y_MMSE_train= minmax_scale(y_MMSE_train)
y_MMSE_test=minmax_scale(y_MMSE_test)

SNP_train = np.concatenate([SNP_train.astype('int')//10,SNP_train.astype('int')%10],axis=1)
SNP_test = np.concatenate([SNP_test.astype('int')//10,SNP_test.astype('int')%10],axis=1)


X_train, X_val, y_train, y_val = train_test_split(
                        np.concatenate([select_X_train,y_MMSE_train ],axis=1), yg_train,
                test_size = 0.20,stratify =yg_train, random_state = random_seed)

SNP_train, SNP_val, y_train2, y_val2 = train_test_split(
                        SNP_train, yg_train,
                test_size = 0.20,stratify =yg_train, random_state = random_seed)

y_MMSE_train = X_train[:,-1]
y_MMSE_val = X_val[:,-1]

X_train = X_train[:,:-1]
X_val = X_val[:,:-1]

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
yg_train = np.concatenate([ad_ge_ADNI,ad_ge_ANM1,ad_ge_ANM2,ad_ge_ROS])
select_X_train=  np.concatenate([ADNI_value.T.values,ANM1_value.T.values,ANM2_value.T.values,ROS_value.T.values])
SNP_train =np.concatenate([ADNI_SNP.values.reshape(-1,1),ANM1_SNP.values.reshape(-1,1),ANM2_SNP.values.reshape(-1,1), ROS_SNP.values.reshape(-1,1)])
y_MMSE_train =np.concatenate([ADNI_MMSE.values.reshape(-1,1),ANM1_MMSE.values.reshape(-1,1),ANM2_MMSE.values.reshape(-1,1), ROS_MMSE.values.reshape(-1,1)])

select_X_test= EW_value.T.values
yg_test= ad_ge_EW
SNP_test= EW_SNP.values.reshape(-1,1)
y_MMSE_test=  EW_MMSE.values.astype('float').reshape(-1,1)

y_MMSE_train= minmax_scale(y_MMSE_train)
y_MMSE_test=minmax_scale(y_MMSE_test)

SNP_train = np.concatenate([SNP_train.astype('int')//10,SNP_train.astype('int')%10],axis=1)
SNP_test = np.concatenate([SNP_test.astype('int')//10,SNP_test.astype('int')%10],axis=1)


X_train, X_val, y_train, y_val = train_test_split(
                        np.concatenate([select_X_train,y_MMSE_train ],axis=1), yg_train,
                test_size = 0.20,stratify =yg_train, random_state = random_seed)

SNP_train, SNP_val, y_train2, y_val2 = train_test_split(
                        SNP_train, yg_train,
                test_size = 0.20,stratify =yg_train, random_state = random_seed)

y_MMSE_train = X_train[:,-1]
y_MMSE_val = X_val[:,-1]

X_train = X_train[:,:-1]
X_val = X_val[:,:-1]

print(y_val)
print(y_val2)


param_grid = {'learningRate' : [0.0005,0.00005,0.000005] , 'weightDecay': [0.0001], 'num_layer': [4,6,8],'batch': [0]} 

best_auc = 0  
best_params = {}
num = 0

X_train = torch.tensor(X_train.astype('float'), dtype=torch.float32).cuda()
X_val = torch.tensor(X_val.astype('float'), dtype=torch.float32).cuda()

y_label_train2 = torch.tensor(y_train.astype('float'), dtype=torch.float32).cuda()
y_MMSE_train = torch.tensor(y_MMSE_train.astype('float'), dtype=torch.float32).cuda()
ds = TensorDataset(X_train,y_MMSE_train)
results = []

for learningRate in param_grid['learningRate']:
    for weightDecay in param_grid['weightDecay']:
        for num_layer in param_grid['num_layer']:
            for batch in param_grid['batch']:
                num_epochs = 10000
                min_eval_auc = 0 
                best_auc = 0 
                patience_limit = 10000 
                patience_check = 0 

                if batch == 0 :
                    loader  = DataLoader(ds, batch_size=1,shuffle=True)
                else:
                    loader  = DataLoader(ds, batch_size=X_train.shape[0],shuffle=True)
                        
                if num_layer == 4 :
                    net = DNN4()
                elif num_layer == 6:
                    net = DNN6()
                else:
                    net = DNN8()
                device = 'cuda' if torch.cuda.is_available() else 'cpu'
                net = net.to(device)
                
                optimizer = optim.SGD(net.parameters(), lr=learningRate, weight_decay=weightDecay)
                loss_fn2 =nn.MSELoss()

                for epoch in (range(num_epochs)):
                    running_loss = 0.0
                    for i, data in enumerate(loader, 0):
                        xg,y1= data
                        output1 = net.forward(xg)

                        net.train()
                        optimizer.zero_grad()
                        if batch == 0 :
                            loss =  loss_fn2(output1[0], y1)
                        else:
                            output1  = output1.squeeze()
                            loss =  loss_fn2(output1, y1)
                        loss.backward(retain_graph=True)
                        optimizer.step()
                    net.eval() 
                    epoch = epoch+1
                    eval_loss = 0
                    if (epoch+1) % 100 == 0 :
                        test1 = net.forward(X_val.clone().detach())
                        print ('Epoch [{}], Loss: {:.4f}'.format(epoch+1 ,loss_fn2(test1.squeeze(), torch.tensor(y_val.astype('float'), dtype=torch.float32).cuda())))
                        eval_loss = loss_fn2(test1.squeeze(), torch.tensor(y_val.astype('float'), dtype=torch.float32).cuda())
                        test1 = test1.cpu().detach().numpy()
                        train1 = net.forward(X_train.clone().detach())
                        train1 = train1.cpu().detach().numpy()
                        model4 = SVC(gamma='auto', probability=True)
                        model4 = model4.fit(train1 , y_label_train2.cpu().detach().numpy())
        
                        print ("  AUC %.3f" %(roc_auc_score(y_val.reshape(-1),model4.predict_proba(test1)[:,1])))
                    test1 = net.forward(X_val.clone().detach())
                    test1 = test1.cpu().detach().numpy()
                    train1 = net.forward(X_train.clone().detach())
                    train1 = train1.cpu().detach().numpy()
                    model4 = SVC(gamma='auto', probability=True)
                    model4 = model4.fit(train1 , y_label_train2.cpu().detach().numpy())
                    eval_auc = roc_auc_score(y_val.reshape(-1),model4.predict_proba(test1)[:,1])
                    if best_auc >= eval_auc: 
                        patience_check += 1

                        if patience_check > patience_limit: 
                            break

                    else: # loss가 개선된 경우
                        best_auc = eval_auc
                        patience_check = 0

                    if min_eval_auc < best_auc:

                        min_eval_auc = eval_auc
                        torch.save(net, '/official/checkpoint_best.pt')
                
                net = torch.load('/official/checkpoint_best.pt')
                test1 = net.forward(X_val.clone().detach())
                test1 = test1.cpu().detach().numpy()
                train1 = net.forward(X_train.clone().detach())
                train1 = train1.cpu().detach().numpy()
                model4 = SVC(gamma='auto', probability=True)
                model4 = model4.fit(train1 , y_label_train2.cpu().detach().numpy())
                print ("Using GE  AUC %.3f" %(roc_auc_score(y_val.reshape(-1),model4.predict_proba(test1)[:,1])))

                auc = roc_auc_score(y_val.reshape(-1),model4.predict_proba(test1)[:,1])
                num=num+1
                print(num)
                results.append({'auc': auc,'learningRate': learningRate, 'weightDecay': weightDecay, 'num_layer': num_layer, 'batch': batch})
                if auc > best_auc:
                    best_auc = auc
                    best_params = {'learningRate': learningRate, 'weightDecay': weightDecay, 'num_layer': num_layer, 'batch': batch}
                
                

print("Best Hyperparameters ml:", best_params)
print("Best AUC:", best_auc)


param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
                

best_auc = 0  
best_params = {}  
num = 0
for C in param_grid['C']:
    for gamma in param_grid['gamma']:
        for kernel in param_grid['kernel']:

            svm_model = SVC(C=C, gamma=gamma, kernel=kernel, probability=True)
            svm_model.fit(SNP_train, y_train)
            y_pred_proba = svm_model.predict_proba(SNP_val)[:, 1]
            auc = roc_auc_score(y_val, y_pred_proba)
            num=num+1
            print(num)
            if auc > best_auc:
                best_auc = auc
                best_params = {'C': C, 'gamma': gamma, 'kernel': kernel}
                

print("Best Hyperparameters ape:", best_params)
print("Best AUC:", best_auc)

[0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0.
 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0.
 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1.
 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0.
 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 

Epoch [7300], Loss: 0.5281
  AUC 0.702
Epoch [7400], Loss: 0.5255
  AUC 0.703
Epoch [7500], Loss: 0.5267
  AUC 0.703
Epoch [7600], Loss: 0.5253
  AUC 0.703
Epoch [7700], Loss: 0.5339
  AUC 0.701
Epoch [7800], Loss: 0.5334
  AUC 0.701
Epoch [7900], Loss: 0.5288
  AUC 0.701
Epoch [8000], Loss: 0.5308
  AUC 0.701
Epoch [8100], Loss: 0.5205
  AUC 0.703
Epoch [8200], Loss: 0.5294
  AUC 0.703
Epoch [8300], Loss: 0.5290
  AUC 0.702
Epoch [8400], Loss: 0.5985
  AUC 0.700
Epoch [8500], Loss: 0.5321
  AUC 0.702
Epoch [8600], Loss: 0.5187
  AUC 0.701
Epoch [8700], Loss: 0.5344
  AUC 0.701
Epoch [8800], Loss: 0.5316
  AUC 0.702
Epoch [8900], Loss: 0.5258
  AUC 0.701
Epoch [9000], Loss: 0.5353
  AUC 0.701
Epoch [9100], Loss: 0.5243
  AUC 0.702
Epoch [9200], Loss: 0.5307
  AUC 0.702
Epoch [9300], Loss: 0.5315
  AUC 0.702
Epoch [9400], Loss: 0.5309
  AUC 0.702
Epoch [9500], Loss: 0.6120
  AUC 0.701
Epoch [9600], Loss: 0.5295
  AUC 0.702
Epoch [9700], Loss: 0.5238
  AUC 0.701
Epoch [9800], Loss: 0.536

In [ ]:
selection_model = SVC( gamma=0.0001, kernel='rbf',C =  0.1, probability=True)
selection_model.fit(X_train, y_train)
ml =   SVC( gamma=0.1, kernel='rbf',C =  0.1, probability=True)
ml.fit(SNP_train, y_train)


print("Using SVM  AUC %.5f" %(roc_auc_score(yg_test,selection_model.predict_proba(select_X_test)[:,1].astype(float))))
print("Using APOE4  AUC %.5f" %(roc_auc_score(yg_test,list(ml.predict_proba(SNP_test)[:,1]))))
print("Ensemble AUC %.5f" %(roc_auc_score(yg_test,list(ss.zscore(selection_model.predict_proba(select_X_test)[:,1].astype(float)) + ss.zscore(ml.predict_proba(SNP_test)[:,1])))))
